In [1]:
# SIMULATED DATA GENERATION
import copy
import math as M
import random
from pairwise_distance import *


def jukesCantor(genome,gen_time):
  """
  Input: genome, a character string consisting of a, c, g, and t
         gen_time, genetic time
  Output: a randomly modified string according to the Juke's cantor model
    The following coupled ODEs are integrated from t=0 to t=gen_time

    dot a = -a + (c + g + t) /3
    dot c = -a + (g + t + a) /3
    dot g = -g + (t + a + c) /3
    dot t = -t + (a + c + g) /3
  
    using the solution 
 
    a(t)=(3/4)*exp(-t)*(a(0)-(c(0)+g(0)+t(0))/3)
        +(1/4)*(a(0)+c(0)+g(0)+t(0))

    and similarly for c(t), g(t), and t(t).

    In other words, if 'a' is the initial base, its persistence
    probability after a time t is 

    3/4 exp(-t) + 1/4

    and likewise for the other three base inputs.

  """
  if type(gen_time) !=  float :
    raise Exception("Time must be a float")
  for b in genome:
    if not ( b=='a' or b=='c' or b=='g' or b=='t' ):
       raise Exception("Invalid input genome")
  persistence_prob=(3.*M.exp(-gen_time)+1.)/4.
  new_genome=[]
  for base in genome:
    p=random.random()
    if p < persistence_prob:
      new_base=base
    else: 
      if base=='a':
        seq=('c','g','t')
      if base=='c':
        seq=('a','g','t')
      if base=='g':
        seq=('a','c','t')
      if base=='t':
        seq=('a','c','g')
      new_base=random.choice(seq)
    new_genome.append(new_base)
  return(new_genome)

# The following routine, which applies mutations randomly, chooses the Jukes-Cantor algorithm,
# but some other model can be used by changing the function below.

def evolve(genome,time):
   return(jukesCantor(genome,time))

def generateHelper(initialGenome,listIn):
 time=listIn[0]
 newGenome=evolve(initialGenome,time)
 listIn[0]=newGenome
 if not ( len(listIn) == 1 or len(listIn) == 3):
    raise Exception("Lists must have one or three elements") 
 if len(listIn) == 1:
   return
 else:
   generateHelper(newGenome,listIn[1])
   generateHelper(newGenome,listIn[2])
   return 

def generateDriver(initialGenome,listIn):
  """
  This algorithm assumes a tree with input of the form:

  [ genTime, leftBranch, rightBranch]

  where the branches can either be tripleton lists of the format above
  or leaves represtented as a single nonnegative number enclosed in a list.
  genTime above is a non-negative real number. A deep copy of the input list is made,
  and the genetic times are replaced with the stochastically evolved genomes.

  An initial genone at the root of the binary tree is required as input, and this
  must be in a format of a list of base charcters, i.e., 'a', 'c', 'g', or 't'.

  """
  listInBis=copy.deepcopy(listIn)
  generateHelper(initialGenome,listInBis)
  return(listInBis)
             
def extract_genomes(tree):
 """
 This function extract the genome on the output of generateDriver
 """
 if len(tree)==1 :
   return(tree)
 else :
   return(extract_genomes(tree[1])+extract_genomes(tree[2]))
 
def rec(parentA,parentB,b):
  """
  This function create a recombinant genome from parentA and parentB
  """
  return(parentA[:b]+parentB[b:])

def hamming_distance(seq1,seq2):
    """
    This function compute the difference between two sequences
    """
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must be of equal length")
    sum=0
    for i in range(len(seq1)):
        if seq1[i]!=seq2[i]:
            sum+=1
    return sum

From here, we generate a genome sequence and do the statistical test

In [3]:
#we define the initial genome as a sequence of a of length 500
initial_gen='a'*500
#here is the input tree of the form [genTime, leftBranch, RightBranch] in this case we have tree of 3 leaves
gen_time=[1.0,[1.0,[1.0],[1.0]],[1.0]]
#we start generating the genome sequences of the tree
tree=generateDriver(initial_gen,gen_time)
#here we extract the sequences generated
seq=extract_genomes(tree)
#let us see how many species do we have
len(seq)

3

In [4]:
seq1=seq[0]
seq3=seq[2]
seq2=seq[1]
#let us see the distance between them. Here d_12 should be short and d_13, d_23 should be long
d_12=hamming_distance(seq1,seq2)
d_13=hamming_distance(seq1,seq3)
d_23=hamming_distance(seq2,seq3)
print('d_12={},d_13={}, d_23={}'.format(d_12,d_13,d_23))

d_12=339,d_13=350, d_23=363


In [5]:
#the recombinant sequence is a recombination of the first and the third sequences
seq_rec=rec(seq1,seq3,200)
#here we compute the distance between the recombinant sequence and its parents
d_1rec=hamming_distance(seq1,seq_rec)
d_3rec=hamming_distance(seq3,seq_rec)
#the time divergent of the 3 species is given here
t_12=2.0
t_13=3.0
t_23=3.0

In [6]:
#let us see the distance between the recombinant sequence and its parents.
print('d_1rec={},d_13={}, d_3rec={}'.format(d_1rec, d_13, d_3rec))

d_1rec=213,d_13=350, d_3rec=137


In [7]:
#here we do the computation of the estimated parameter lambda and the time divergence
lambda_hat=(d_1rec+d_13+d_3rec)/(2*(t_12+t_13+t_23))
t_1rec_hat=(d_1rec*(t_12+t_13+t_23)/(2*(d_1rec+d_13+d_3rec)))
t_13_hat=(d_13*(t_12+t_13+t_23)/(2*(d_1rec+d_13+d_3rec)))
t_3rec_hat=(d_3rec*(t_12+t_13+t_23)/(2*(d_1rec+d_13+d_3rec)))

In [8]:
#let us see the inferred distance
print('d_1rec_hat={},d_13_hat={}, d_3rec_hat={}'.format(2*lambda_hat*t_1rec_hat, 2*lambda_hat*t_13_hat, 2*lambda_hat*t_3rec_hat))

d_1rec_hat=106.5,d_13_hat=175.0, d_3rec_hat=68.5


In [9]:
import math as mt

In [10]:
#we compute the Z-score
Z=(2*lambda_hat*abs(t_13_hat-t_3rec_hat))/(mt.sqrt(2*lambda_hat*(t_13_hat+t_3rec_hat)))

In [11]:
#we could see here that Z is greater than 1.96 which is the value of Z when alpha=0.05 so we could reject the null hypothesis (No recombination)
Z

6.824960229592816

In [35]:
#this is to see the value of Z to the data with no recombination
lambda_hat_norec=(d_12+d_13+d_23)/(2*(t_12+t_13+t_23))
t_13_hat_norec=(d_13*(t_12+t_13+t_23)/(2*(d_12+d_13+d_23)))
t_23_hat=(d_23*(t_12+t_13+t_23)/(2*(d_12+d_13+d_23)))
Z_norec=(2*lambda_hat_norec*abs(t_13_hat_norec-t_23_hat))/(mt.sqrt(2*lambda_hat_norec*abs(t_13_hat_norec+t_23_hat)))

In [37]:
#the value of Z here is less than 1.96
Z_norec

0.3442576418660435